In [3]:
import newspaper
from newspaper import Article, ArticleException
from newspaper.configuration import Configuration
from pathlib import Path
import pandas as pd
import numpy as np

# parallelisation functionality
from multiprocess import Pool
import psutil
from tqdm import tqdm

In [28]:
config = Configuration()
config.request_timeout = 10

In [29]:
src = "../../data/articles/"
fname = "url_list_for_article_scraping.csv.gzip"
urls = pd.read_csv(Path(src, fname), compression="gzip", usecols=["domain", "url"])

In [47]:
! ls ../../data_2022-02-10/articles


article_corpus_clean.csv.gzip
article_corpus_clean_honesty_component_scores_glove.csv.gzip
article_corpus_raw.rds.gz
articles.csv.gzip
old
OLS_predictions_articles.csv
url_independent_scores.csv.gzip
url_list_for_article_scraping.csv.gzip
url_NG_scores.csv.gzip


In [48]:
collected_articles = pd.read_csv("../../data_2022-02-10/articles/article_corpus_clean.csv.gzip", compression="gzip")

In [49]:
collected_articles.head()

,url,Score,link_text,remove,wc,party,fishy
0,https://www.washingtonpost.com/politics/2018/1...,100.0,"\n\nDays before the midterm elections, Preside...",keep,775,Democrat,non_fishy
1,https://apnews.com/eb82c8597cef4c95a473e525402...,95.0,Residents look at a home damaged by a magnitud...,keep,715,Democrat,non_fishy
2,https://www.usatoday.com/story/news/nation/201...,100.0,Federal judge blocks Trump from deporting hund...,keep,1116,Democrat,non_fishy
3,https://www.cnn.com/2018/10/03/politics/senate...,80.0,Washington (CNN) The Senate on Wednesday passe...,keep,144,Democrat,non_fishy
4,https://www.usatoday.com/story/travel/flights/...,100.0,Senate approves bill that would regulate airli...,keep,506,Democrat,non_fishy


In [52]:
remaining_urls = urls[~urls["url"].isin(collected_articles["url"])]

In [55]:
remaining_urls.to_csv("../../data/articles/remaining_urls.csv.gzip", compression="gzip", index=False)

In [53]:
remaining_urls

,domain,url
5,vindy.com,https://www.vindy.com/news/latest-news/2022/12...
12,thehill.com,https://thehill.com/opinion/congress-blog/3780...
13,prnewswire.com,https://www.prnewswire.com/news-releases/study...
23,radio.foxnews.com,https://radio.foxnews.com/2022/12/01/common-gr...
25,foxbusiness.com,https://www.foxbusiness.com/politics/gop-lawma...
...,...,...
2885027,nytimes.com,https://www.nytimes.com/2014/08/29/business/en...
2885037,pleasantonexpress.com,http://www.pleasantonexpress.com/news/2014-04-...
2885042,politico.com,http://www.politico.com/story/2013/09/farm-bil...
2885043,politi.co,http://politi.co/17Tca4n


In [30]:
excluded_domains = ["twitter.com", "youtube.com", "facebook.com",
            "instagram.com", "cards.twitter.com", "google.com", "yahoo.com", 
            "", "bit.ly", "ow.ly", "fb.me", "eventbrite.com", "medium.com", "twitpic.com",
            "connectiontimeouttohosthttp:"]
urls = urls[~urls["domain"].isin(excluded_domains)]

In [31]:
urls = urls.dropna()
urls = urls[urls["domain"].apply(lambda x: ".gov" not in x)]
urls = urls.drop_duplicates()

In [32]:
len(urls)

729393

In [33]:
def get_link_text(url):
    try:
        article = Article(url, config=config)
        article.download()
        article.parse()
    except Exception as e:
        article np.nan
    batch.loc[url, "link_text"] = article

In [34]:
N_batches = 10
batch_size = int(len(urls) / N_batches) 
url_batches = [urls[i * batch_size : (i + 1) * batch_size].copy() for i in range(N_batches)]

In [45]:
batch

,domain,link_text
url,,
https://www.vindy.com/news/latest-news/2022/12/yars-selected-as-preferred-site-for-8-c-130j-aircraft/,vindy.com,NaN
https://thehill.com/opinion/congress-blog/3780323-the-domino-effect-of-president-bidens-war-on-oil-and-gas/,thehill.com,NaN
https://www.prnewswire.com/news-releases/study-finds-cumulative-97-8-billion-of-shale-investment-in-ohio-through-2021--second-half-of-2021-brings-in-2-5-billion-in-investment-301704884.html,prnewswire.com,NaN
https://radio.foxnews.com/2022/12/01/common-ground-meeting-in-the-middle-on-energy/,radio.foxnews.com,NaN
https://www.foxbusiness.com/politics/gop-lawmakers-blast-biden-turning-venezuelan-dictator-oil-while-curbing-domestic-production,foxbusiness.com,NaN
...,...,...
https://www.durbinforsenate.com,durbinforsenate.com,NaN
https://thegrio.com/2020/07/16/politicking-with-sen-dick-durbin/,thegrio.com,NaN
https://taskandpurpose.com/news/vanessa-guillen-military-investigations-durbin,taskandpurpose.com,NaN


In [44]:
dst = "../../data/articles"
fname = "article_corpus_raw_{}.csv.gzip"

number_of_cores = 20
pool = Pool(number_of_cores)

i = 0
batch = url_batches[i]
batch = batch.set_index("url")
batch["link_text"] = np.nan

for link_text in tqdm(pool.imap_unordered(func=get_link_text, iterable=batch.index), total=len(batch)):
    pass

# turn off parallel workers 
pool.close()

#batch["link_text"] = link_texts
batch.to_csv(Path(dst, fname.format(i)), compression="gzip", index=False)


 19%|█████████████████████████████████████████████▊                                                                                                                                                                                                 | 13998/72939 [10:48<31:41, 31.00it/s]/home/jana/anaconda3/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:845: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 2. 
  warnings.warn(str(msg))
 28%|██████████████████████████████████████████████████████████████████▍                                                                                                                                                                          | 20433/72939 [16:16<1:49:15,  8.01it/s]/home/jana/anaconda3/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:845: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 2. 
  warnings.warn(str(msg))
 34%|█████████████████████████████████████████████████████████████████████████

KeyboardInterrupt: 

Process ForkPoolWorker-159:
Process ForkPoolWorker-144:
Process ForkPoolWorker-145:
Process ForkPoolWorker-158:
Process ForkPoolWorker-152:
Process ForkPoolWorker-160:
Process ForkPoolWorker-151:
Process ForkPoolWorker-154:
Process ForkPoolWorker-141:
Process ForkPoolWorker-148:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/jana/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/home/jana/anaconda3/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
Process ForkPoolWorker-157:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call 